In [10]:
import pickle
from nltk.util import ngrams
from nltk import word_tokenize


In [9]:
eng_unigrams = pickle.load(open('eng_unigram_count.p', 'rb'))
eng_bigrams = pickle.load(open('eng_bigram_count.p', 'rb'))
ital_unigrams = pickle.load(open('ital_unigram_count.p', 'rb'))
ital_bigrams = pickle.load(open('ital_bigram_count.p', 'rb'))
fr_unigrams = pickle.load(open('fr_unigram_count.p', 'rb'))
fr_bigrams = pickle.load(open('fr_bigram_count.p', 'rb'))


In [12]:
def compute_prob(text, unigram_dict, bigram_dict):
    # V is the vocabulary size in the training data (unique tokens)
    V =  len(unigram_dict)
    unigrams_test = word_tokenize(text)
    bigrams_test = list(ngrams(unigrams_test, 2))
    
    p_laplace = 1  # calculate p using Laplace smoothing

    for bigram in bigrams_test:
        n = bigram_dict[bigram] if bigram in bigram_dict else 0
        d = unigram_dict[bigram[0]] if bigram[0] in unigram_dict else 0
        p_laplace = p_laplace * ((n + 1) / (d + V))

    return p_laplace

In [23]:
#open test files
tests = open("LangId.test.txt", 'r')
solutions = open("LangId.sol.txt", 'r')
results = open("LangId.results.txt", 'w')
incorrect_guesses = []
test_cases = []

#read in each test case line by line
for x in range(1,301):
    test_cases.append((tests.readline().rstrip('\n'), solutions.readline()))

#Calculate the probabilities for each language and compare the best guess to the test case solution
for case in test_cases:
    eng_prob = compute_prob(case[0], eng_unigrams, eng_bigrams)
    ital_prob = compute_prob(case[0], ital_unigrams, ital_bigrams)
    fr_prob = compute_prob(case[0], fr_unigrams, fr_bigrams)
    guess = ""
    if eng_prob > ital_prob and eng_prob > fr_prob:
        guess = "English\n"
    elif fr_prob > ital_prob:
        guess = "French\n"
    else:
        guess = "Italian\n"
    results.write(f"{guess}")
    solution = case[1].split(" ")
    if guess != solution[1]:
        incorrect_guesses.append(solution[0])
        
#calculating accuracy and printing it and outputing the incorrect guesses
print(f"Accuracy: {(1-len(incorrect_guesses)/300)*100}%")
print("Incorrect cases: " ,end="")
for x in incorrect_guesses:
    print(f"{x}, ", end="")



Accuracy: 97.0%
Incorrect cases: 24, 44, 87, 191, 220, 244, 262, 279, 297, 